# Performance Test of various data frame persistance methods

In [1]:
dataset_size = 5000

In [2]:
import random
import string
import datetime
import time
import pandas as pd
from pathlib import Path
import plotly.express as pe

C:\Users\v.dekanovsky\Anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [3]:
def get_random_string(length: int) -> str:
    """Generated random string up to the specific lenght"""
    
    letters = string.ascii_letters
    result_str = ''.join([random.choice(letters) for i in range(random.randint(3,length))])
    return result_str

def get_random_sentence(words: int=5) -> str:
    """Generate random sentence - list of random works"""
    
    words = []
    for i in range(5):
        words.append(get_random_string(10)[:random.randint(0,10)])
    return " ".join(words)

def random_date(start: str, end: str, format: str) -> str:
    """Generate random date of specified format"""
    
    earliest = datetime.datetime.strptime(start, format)
    latest  = datetime.datetime.strptime(end, format)
    delta = latest - earliest
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds    
    random_second = random.randrange(int_delta)
    return (earliest + datetime.timedelta(seconds = random_second)).strftime(format)

In [4]:
# https://numpy.org/doc/stable/user/basics.types.html
data = []
for i in range(dataset_size):
    data.append(
        [random.randint(-127,127),  # int8
         random.randint(-32768,32767),  # int16
         random.randint(-2147483648,2147483647),  # int32
         random.randint(-9223372036854775808 ,9223372036854775807),  # int64
         random.randint(0,255),  # uint8
         round(random.uniform(0,10000),2),
         round(random.uniform(0,1000000),2),
         get_random_string(10),
         get_random_sentence(5),
         random.choice([get_random_string(10) for i in range(25)]),
         random_date("1900-01-01","2020-05-01","%Y-%m-%d"),
         random_date("1900-01-01T00:00:00","2020-05-01T23:59:59","%Y-%m-%dT%H:%M:%S"),
         random.choice([True,False])])

In [5]:
# create data frame from the list of generated random values

column_names = ["Int8", "Int16", "Int32", "Int64", "UInt8", "Float32", "Float64", 
                "String", "Sentence", "Category", "Date", "DateTime", "Bool"]
df = pd.DataFrame(data, columns=column_names)
df["Int8"] = df["Int8"].astype("int8")
df["Int16"] = df["Int16"].astype("int16")
df["Int32"] = df["Int32"].astype("int32")
df["UInt8"] = df["UInt8"].astype("uint8")
df["Float32"] = df["Float32"].astype("float32")
df["Category"] = df["Category"].astype("category")
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
df["DateTime"] = pd.to_datetime(df["DateTime"], format="%Y-%m-%dT%H:%M:%S")
df["DateTime+Zone"] = pd.to_datetime(df["DateTime"], format="%Y-%m-%dT%H:%M:%S")
df["TimeDelta"] = df["DateTime"]-df["Date"]
#df["DateTime+Zone"] = df["DateTime+Zone"].dt.tz_localize('Europe/Vienna')

# store for size comparison
df.to_csv("benchmark.csv", index=False)
orig_size = Path("benchmark.csv").stat().st_size

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
Int8             5000 non-null int8
Int16            5000 non-null int16
Int32            5000 non-null int32
Int64            5000 non-null int64
UInt8            5000 non-null uint8
Float32          5000 non-null float32
Float64          5000 non-null float64
String           5000 non-null object
Sentence         5000 non-null object
Category         5000 non-null category
Date             5000 non-null datetime64[ns]
DateTime         5000 non-null datetime64[ns]
Bool             5000 non-null bool
DateTime+Zone    5000 non-null datetime64[ns]
TimeDelta        5000 non-null timedelta64[ns]
dtypes: bool(1), category(1), datetime64[ns](3), float32(1), float64(1), int16(1), int32(1), int64(1), int8(1), object(2), timedelta64[ns](1), uint8(1)
memory usage: 584.9+ KB


In [6]:
# define the types of transformation to use
exporting_types = {
    "csv": {
        "type": "CSV",
        "extension": ".csv",
        "write_function": pd.DataFrame.to_csv,
        "write_params": {"index": False},
        "read_function": pd.read_csv,
        "read_params": {}
    },
    "csv_zip": {
        "type": "CSV zip",
        "extension": ".zip",
        "write_function": pd.DataFrame.to_csv,
        "write_params": {"index": False, "compression": "zip"},
        "read_function": pd.read_csv,
        "read_params": {"compression": "zip"}
    },
    "picklea": {
        "type": "Pickle bz2",
        "extension": ".pkl.bz2",
        "write_function": pd.DataFrame.to_pickle,
        "write_params": {"compression": "bz2"},
        "read_function": pd.read_pickle,
        "read_params": {"compression": "bz2"}
    },
    "pickleb": {
        "type": "Pickle gzip",
        "extension": ".pkl.gzip",
        "write_function": pd.DataFrame.to_pickle,
        "write_params": {"compression": "gzip"},
        "read_function": pd.read_pickle,
        "read_params": {"compression": "gzip"}
    },
    "picklec": {
        "type": "Pickle zip",
        "extension": ".pkl.zip",
        "write_function": pd.DataFrame.to_pickle,
        "write_params": {"compression": "zip"},
        "read_function": pd.read_pickle,
        "read_params": {"compression": "zip"}
    },
    "pickled": {
        "type": "Pickle infer",
        "extension": ".pkl",
        "write_function": pd.DataFrame.to_pickle,
        "write_params": {},
        "read_function": pd.read_pickle,
        "read_params": {}
    },
    "picklee": {
        "type": "Pickle xz",
        "extension": ".pkl.xz",
        "write_function": pd.DataFrame.to_pickle,
        "write_params": {"compression": "xz"},
        "read_function": pd.read_pickle,
        "read_params": {"compression": "xz"}
    },
    "parquet_pyarrow": {
        "type": "Parquet via PyArrow",
        "extension": ".parquet.gzip",
        "write_function": pd.DataFrame.to_parquet,
        "write_params": {},
        "read_function": pd.read_parquet,
        "read_params": {}
    },
    "parquet_fastparquet": {
        "type": "Parquet via fastparquet",
        "extension": ".parquet.gzip",
        "write_function": pd.DataFrame.to_parquet,
        "write_params": {"engine":"fastparquet","compression":"GZIP"},
        "read_function": pd.read_parquet,
        "read_params": {"engine":"fastparquet"}
    },    
    "Hdf5f": {
        "type": "Hdf5 fixed",
        "extension": ".h5",
        "write_function": pd.DataFrame.to_hdf,
        "write_params": {"key":"df", "format":"fixed"},
        "read_function": pd.read_hdf,
        "read_params": {"key":"df"}
    },
    "Hdf5t": {
        "type": "Hdf5 table",
        "extension": ".h5",
        "write_function": pd.DataFrame.to_hdf,
        "write_params": {"key":"df", "format":"table"},
        "read_function": pd.read_hdf,
        "read_params": {"key":"df"}
    },
    "Hdf5c": {
        "type": "Hdf5 compressed",
        "extension": ".h5",
        "write_function": pd.DataFrame.to_hdf,
        "write_params": {"key":"df", "format":"table", "complevel": 9, "complib": "bzip2"},
        "read_function": pd.read_hdf,
        "read_params": {"key":"df"}
    },
    "Excel": {
        "type": "Excel",
        "extension": ".xlsx",
        "write_function": pd.DataFrame.to_excel,
        "write_params": {"index":False},
        "read_function": pd.read_excel,
        "read_params": {}
    },
    # to_sql and read_sql have too much different syntax
    #"SQL": {
    #    "type": "SQL",
    #    "extension": "",
    #    "write_function": pd.DataFrame.to_sql,
    #    "write_params": {"name": "pandas", "con": engine},
    #    "read_function": pd.read_sql,
    #    "read_params": {"name": "pandas", "con": engine}
    #},
    # feather file gets blocked by ArrowIO after the first run
    "Feather": {
        "type": "Feather",
        "extension": ".f",
        "write_function": pd.DataFrame.to_feather,
        "write_params": {},
        "read_function": pd.read_feather,
        "read_params": {}
    }
}

In [7]:
def repeat(f: dict, repetitions: int, df: pd.DataFrame, file: str) -> dict:
    """ Perform an operation specified by input dict number of repetition times on the data frame"""
    
    writes_elapsed = []
    reads_elapsed = []
    for r in range(repetitions):
        
        # write
        start_time = time.time()
        
        # first parameter is self (the dataframe), second the path and then **kwargs
        f["write_function"](df, file, **f["write_params"])
        write_elapsed = time.time() - start_time
        writes_elapsed.append(write_elapsed)
        
        # read
        start_time = time.time()
        df_loaded = f["read_function"](file, **f["read_params"])
        read_elapsed = time.time() - start_time
        reads_elapsed.append(read_elapsed)
    
    return {"write": sum(writes_elapsed)/len(writes_elapsed),
           "read": sum(reads_elapsed)/len(reads_elapsed),
           "df_loaded": df_loaded}

In [8]:
def statistics(df: pd.DataFrame, df_loaded: pd.DataFrame, new_file: str, 
               tp: str, benchmark: dict, out: dict, orig_size: int) -> pd.DataFrame:
    """Generate statistics based on performance test results"""
    
    comparison_df = pd.DataFrame({"orig": df.dtypes, "new": df_loaded.dtypes})
    comparison_df["same"] = comparison_df["orig"]==comparison_df["new"]
    #mismatched_types = comparison_df[comparison_df["same"]==False].index.to_list()
    
    new_size = Path(new_file).stat().st_size
    
    return pd.DataFrame({
    "dtype_preservation": comparison_df["same"].sum()/comparison_df["same"].count(),
    "compression": new_size/orig_size,
    "write_time": out["write"]/benchmark["write"],
    "read_time": out["read"]/benchmark["read"],
    #"mismatched_types": mismatched_types
    }, 
        index=[tp]), comparison_df

In [9]:
def performance_test(exporting_types: dict, df: pd.DataFrame, 
                     out_file: str="out", repetitions: int = 7) -> pd.DataFrame:
    
    """Run performance test for predefined dict of operations"""

    results = []
    for k,v in exporting_types.items():

        # create a file name
        new_file = out_file + v["extension"]
        
        try:
            # repeat the writing and reading several times
            out = repeat(v, repetitions, df, new_file)

            # CSV is the first one and it's set as benchmark for reading and writing times
            if v["type"] == "CSV":
                benchmark = out
                df.to_csv("benchmark.csv", index=False)
                orig_size = Path("benchmark.csv").stat().st_size

            # process the results - dtypes_preservation, compression, write and read_time
            results.append(statistics(df, out["df_loaded"], new_file, v["type"], benchmark, out, orig_size)[0])
        
        except Exception as e:
            print(f"{k} failed - {e}")

    return pd.concat(results)

In [10]:
# performance test
performance_df = performance_test(exporting_types, df)

# results
performance_df.style.format("{:.2%}")

parquet_pyarrow failed - ('Unsupported numpy type 22', 'Conversion failed for column TimeDelta with type timedelta64[ns]')
Hdf5f failed - Cannot store a category dtype in a HDF5 dataset that uses format="fixed". Use format="table".
Feather failed - ('Unsupported numpy type 22', 'Conversion failed for column TimeDelta with type timedelta64[ns]')


,dtype_preservation,compression,write_time,read_time
CSV,33.33%,100.00%,100.00%,100.00%
CSV zip,33.33%,48.30%,136.77%,137.51%
Pickle bz2,100.00%,44.04%,37.22%,140.98%
Pickle gzip,100.00%,44.47%,25.71%,38.20%
Pickle zip,100.00%,44.48%,20.51%,40.64%
Pickle infer,100.00%,55.01%,1.91%,24.70%
Pickle xz,100.00%,38.01%,123.95%,168.34%
Parquet via fastparquet,100.00%,43.66%,192.23%,330.38%
Hdf5 table,100.00%,25818.87%,38.61%,173.34%
Hdf5 compressed,100.00%,25819.12%,87.69%,311.56%


In [ ]:
# display the graph with the results
fig = pe.bar(performance_df.T, barmode='group', text="value")
    
fig.update_traces(texttemplate='%{text:.2%}', textposition='auto')
fig.update_layout(title=f"Statistics for {dataset_size} records")
fig.show()

In [ ]:
fig = pe.bar(performance_df.loc[["CSV","Pickle infer","Parquet via fastparquet", "Hdf5 table", "Excel"]].T, 
             barmode='group', text="value")
fig.update_traces(texttemplate='%{text:.2%}', textposition='auto')
fig.update_layout(title=f"Statistics for {dataset_size} records - performance test 7 repetititons", 
                  yaxis={"type": "log", "title": "value % (log scale)"})
fig.show()